# Creating Customm Tests

In [1]:
## Replace the code below with the code snippet from your project ## 

import validmind as vm

vm.init(
    api_host = "https://api.prod.validmind.ai/api/v1/tracking",
    api_key = "...",
    api_secret = "...",
    project = "..."
)

2023-09-08 17:15:36,629 - INFO(validmind.api_client): Connected to ValidMind. Project: Customer Churn - Initial Validation (clkvhtg6g0005q08h5h9uhtjl)


## Implementing Custom Metrics

It is possible to implement custom metrics or threshold test classes. The are only two requirements for getting this to work:

- We need to build a `TestPlan` that can execute the custom metric or threshold test (or add it to an existing `TestPlan`, TBD).
- We need to implement a `run` method on the custom metric or threshold test class.

### Implementing a Custom Metric

The following example shows how to implement a custom metric that calculates the mean of a list of numbers.

In [2]:
from dataclasses import dataclass
from validmind.vm_models import Metric

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)
        
        return self.cache_results(mean)

### Testing the Custom Metric

It is possible to run a custom metric without running an entire test suite. This is useful for testing the metric before integrating it into a test suite.

The key idea is to create a `TestContext` object and pass it to the metric initializer. When a test suite is executed, the `TestContext` is created by the `TestSuite` class and passed down to every associated metric and threshold test. However, when we want to test a metric in isolation, we need to create the `TestContext` ourselves.

In this example we don't need to pass any arguments to the `TestContext` initializer, but it is possible to pass any arguments as required by `required_inputs`.

In [3]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestSuiteMetricResult(result_id="mean_of_values", metric, figures)

We can also inspect the results of the metric by accessing the `result` variable:

In [4]:
mean_metric.result.show()

### Add a `summary()` Method to the Custom Metric

In [5]:
from dataclasses import dataclass

import pandas as pd
from validmind.vm_models import Metric, ResultSummary, ResultTable, ResultTableMetadata

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def summary(self, metric_value):
        # Create a dataframe structure that can be rendered as a table
        simple_df = pd.DataFrame({"Mean of Values": [metric_value]})

        return ResultSummary(
            results=[
                ResultTable(
                    data=simple_df,
                    metadata=ResultTableMetadata(title="Example Table"),
                ),                
            ]
        )        
        
    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)
        return self.cache_results(mean)

In [6]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestSuiteMetricResult(result_id="mean_of_values", metric, figures)

In [7]:
mean_metric.result.show()

### Add Figures to a Metric

In [8]:
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from validmind.vm_models import Figure, Metric, ResultSummary, ResultTable, ResultTableMetadata

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def summary(self, metric_value):
        # Create a dataframe structure that can be rendered as a table
        simple_df = pd.DataFrame({"Mean of Values": [metric_value]})

        return ResultSummary(
            results=[
                ResultTable(
                    data=simple_df,
                    metadata=ResultTableMetadata(title="Example Table"),
                ),
            ]
        )        

        
    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)

        # Create a random histogram with matplotlib
        fig, ax = plt.subplots()
        ax.hist(np.random.randn(1000), bins=20, color="blue")
        ax.set_title("Histogram of random numbers")
        ax.set_xlabel("Value")
        ax.set_ylabel("Frequency")

        # Do this if you want to prevent the figure from being displayed
        plt.close("all")
        
        figure = Figure(
            for_object=self,
            key=self.key,
            figure=fig
        )

        return self.cache_results(mean, figures=[figure])

In [9]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestSuiteMetricResult(result_id="mean_of_values", metric, figures)

In [10]:
mean_metric.result.show()

In [11]:
from validmind.vm_models import TestSuite

class MyCustomTestSuite(TestSuite):
    """
    Custom test suite
    """

    suite_id = "my_custom_test_suite"
    tests = [MeanMetric]

my_custom_test_suite = MyCustomTestSuite(config={
    "mean_of_values": {
        "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
})
my_custom_test_suite.run()